In [58]:
import pyparsing as pp


In [59]:
# datastructures for C++ syntax

class TypeArgs:
    CONST_TYPE = 1
    VOLATILE_TYPE = 2
    STATIC_TYPE = 4

class CppComplexTypeDefinition:
    DEFAULT = 0
    CLASS   = 1
    STRUCT  = 2
    UNION   = 3
    
    VISIBILITY_DEFAULT = 0
    VISIBILITY_PRIVATE = 1
    VISIBILITY_PROTECTED = 2
    VISIBILITY_PUBLIC = 3
    
    def __init__(self, complex_type, name, base_types=[], member_variables=[], member_functions=[]):
        self.complex_type = complex_type
        self.name = name
        self.base_types = base_types
        self.member_variables = member_variables
        self.member_functions = member_functions
    

class FunctionArgs:
    CONST_FUNCTION = 1
    VIRTUAL_FUNCTION = 2
    CONSTRUCTOR_FUNCTION = 4
    DESTRUCTOR_FUNCTION = 8
    ABSTRACT_FUNCTION = 16
    INLINE_FUNCTION = 32

class CppTypeExpression:
    def __init__(self, type_name, type_args=0, template_args=[], refs=[]):
        self.type_args = type_args
        self.type_name = type_name
        # if this type is a template type, this will contain the template argument values
        self.template_args = template_args

class CppPointerTypeExpression:
    POINTER_VAR   = 1
    REFERENCE_VAR = 2
    
    def __init__(self, inner_type, ref_type, ref_volatility):
        self.inner_type = inner_type
        self.ref_type = ref_type
        self.ref_volatility = ref_volatility

class CppTypeDefinition:
    def __init__(self, type_expr, type_name):
        self.type_expr = type_expr
        self.type_name = type_name

class CppVarDeclaration:
    def __init__(self, data_type, identifier):
        self.data_type = data_type
        self.identifier = identifier

class CppInheritance:
    def __init__(self, base_class_id, visibility=0):
        self.vis = visibility
        self.base_id = base_class_id

class CppMember:
    def __init__(self, member_decl, visibility=CppComplexTypeDefinition.VISIBILITY_DEFAULT):
        self.member_decl = member_decl
        self.vis = visibility
    
class CppFunctionDeclaration:
    def __init__(self, name, return_type, params, args):
        self.name = name
        self.return_type = return_type
        self.params = params
        self.args = args
    
    def isAbstract(self):
        return bool(self.args & FunctionArgs.ABSTRACT_FUNCTION)
    
    def isConstant(self):
        return bool(self.args & FunctionArgs.CONST_FUNCTION)
    
    def isConstructor(self):
        return bool(self.args & FunctionArgs.CONSTRUCTOR_FUNCTION)
    
    def isDestructor(self):
        return bool(self.args & FunctionArgs.DESTRUCTOR_FUNCTION)
    
    def isInline(self):
        return bool(self.args & FunctionArgs.INLINE_FUNCTION)
    
    def isVirtual(self):
        return bool(self.args & FunctionArgs.VIRTUAL_FUNCTION)
    
class CppFunctionDefinition(CppFunctionDeclaration):
    def __init__(self, decl):
        CppFunctionDeclaration.__init__(self, decl.name, decl.return_type, decl.params, decl.args)

class CppClass:
    def __init__(self, name, fields=[], member_functions=[], inheritances=[]):
        self.name = name
        self.fields = fields
        self.member_functions = member_functions
        self.inheritances = inheritances


In [121]:
# printing C++ code
# (conversion of the data-structures back to code)

class CppPrinter:
    def declaration_str(self, declaration):
        return self.type_expr_str(declaration.data_type) + ' ' + self.identifier_str(declaration.identifier)
    
    def constructor_str(self, constructor):
        c = constructor
        return self.type_expr_str(c.name) + ' ' + self.parameter_list_str(c.params)
    
    def function_decl_str(self, function_decl):
        #print('Debug: ' + str(function_decl))
        res = []
        
        if function_decl.isInline():
            res.append('inline')
        if function_decl.isVirtual():
            res.append('virtual')
        
        rts = self.type_expr_str(function_decl.return_type)
        res.append( rts if not function_decl.isDestructor() else ('~' + rts) )
        
        if function_decl.name is not None:
            res.append(function_decl.name)
        
        res.append(self.parameter_list_str(function_decl.params))
        
        if function_decl.isConstant():
            res.append('const')
        
        if function_decl.isAbstract():
            res.append('= 0')
        
        return ' '.join(res)
    
    def identifier_str(self, identifier):
        return identifier
    
    def inheritance_str(self, inheritance):
        return visibility_str(inheritance.vis) + ' ' + inheritance.base_id
    
    def parameter_list_str(self, pl):
        return '(' + ', '.join(map(self.declaration_str, pl)) + ')'
    
    def type_expr_str(self, type_id):
        if type(type_id) == CppPointerTypeExpression:
            ref_str = '*' if type_id.ref_type == CppPointerTypeExpression.POINTER_VAR   else \
                      '&' if type_id.ref_type == CppPointerTypeExpression.REFERENCE_VAR else \
                      ''
            it = type_id.inner_type
            return self.type_expr_str(it) + (' ' if type(it) != CppPointerTypeExpression else '') + ref_str \
                 + self.volatility_str(type_id.ref_volatility)
        elif type(type_id) == CppTypeExpression:
            #print('DEBUG: %s - %s' % (type_id.type_name, str(type_id.template_args)))
            res = []
            if type_id.type_args & TypeArgs.STATIC_TYPE:
                res.append('static')
            if type_id.type_args & TypeArgs.CONST_TYPE:
                res.append('const')
            if type_id.type_args & TypeArgs.VOLATILE_TYPE:
                res.append('volatile')

            # format template args
            template_str = ''
            if type_id.template_args:
                template_str = '<' + ', '.join(map(self.type_expr_str, type_id.template_args)) + '>'

            res.append(type_id.type_name + template_str)

            return ' '.join(res)
        else:
            raise ValueError(str(type(type_id)) + ' is not a c++ type object!')

    def typedef_str(self, typedef):
        return 'typedef' + ' ' + self.type_expr_str(typedef.type_expr) + ' ' + typedef.type_name
    
    def visibility_str(self, visibility):
        return 'private'   if visibilty == CppComplexTypeDefinition.VISIBILITY_PRIVATE   else \
               'public'    if visibilty == CppComplexTypeDefinition.VISIBILITY_PUBLIC    else \
               'protected' if visibilty == CppComplexTypeDefinition.VISIBILITY_PROTECTED else \
               ''
    
    def volatility_str(self, volatility):
        return 'const'    if volatility == TypeArgs.CONST_TYPE    else \
               'volatile' if volatility == TypeArgs.VOLATILE_TYPE else \
               ''

    def complex_type_prefix_str(self, complex_type_prefix):
        return 'struct' if complex_type_prefix == CppComplexTypeDefinition.STRUCT else \
               'class'  if complex_type_prefix == CppComplexTypeDefinition.CLASS  else \
               'union'  if complex_type_prefix == CppComplexTypeDefinition.UNION  else \
               ''
    
    def complex_type_str(self, complex_type):
        prefix_str = self.complex_type_prefix_str(complex_type.complex_type)
        name_str = complex_type.name
        return prefix_str + ' ' + name_str + ' ' + '{}'
    

In [61]:
# match unparsed scopes
def get_scope(start, end):
    scope = pp.Forward()
    scope <<= pp.Literal(start) \
            + pp.ZeroOrMore(pp.Word(pp.printables.replace(start,'').replace(end,'')) | scope) \
            + pp.Literal(end)
    return scope

def get_separated_list(sep, expr, min_len=0):
    if min_len == 0:
        return pp.Optional(expr + pp.ZeroOrMore(sep + expr))
    elif min_len == 1:
        return expr + pp.ZeroOrMore(sep + expr)
    else:
        return expr + (sep + expr) * (min_len-1) + pp.ZeroOrMore(sep + expr)

def csl(expr, min_len=0):
    return get_separated_list(pp.Literal(',').suppress(), expr, min_len=min_len)


In [62]:
def print_args(function):
    def wrapper(*args, **kwargs):
        printer = CppPrinter()
        
        print('args: %s, kwargs: %s' % (str(args), str(kwargs)))
        for arg in args:
            if type(arg) == CppTypeExpression or type(arg) == CppPointerTypeExpression:
                print(printer.type_expr_str(arg))
        return function(*args, **kwargs)
    return wrapper


In [117]:
# builder functions for the more complex types

def build_type_expression(tokens):
    name = tokens.name[0]
    args = tokens.args
    templates = [tokens.template[0]] if tokens.template else []
    
    return CppTypeExpression(name, args, templates)

def build_pointer_type_expression(inner_type, refs_list):
    printer = CppPrinter()
    #print('building pointer type: %s, %s' % (printer.type_expr_str(inner_type), str(refs_list)))
    
    if not refs_list:
        return inner_type
    el = refs_list.pop(0)
    
    ref_type = el[0]
    if len(el) > 1:
        ref_vol = el[1]
    else:
        ref_vol = 0
    
    return build_pointer_type_expression(
        CppPointerTypeExpression(inner_type, ref_type, ref_vol),
        refs_list
    )

def build_declaration_list(res):
    return [
        CppVarDeclaration(
            build_pointer_type_expression( res.type_id[0], elem.refs ),
            elem.name
        ) for elem in res.ids
    ]

def build_function( parse_result ):
    # build args bitmap
    args = 0
    if parse_result.abstract:
        # we need to dereference here, as 'abstract_functions' is a combination of expressions
        # pyparsing is not context free here
        args += parse_result.abstract[0]
    if parse_result.const:
        args += parse_result.const
    if parse_result.constructor:
        args += FunctionArgs.CONSTRUCTOR_FUNCTION
    if parse_result.destructor:
        args += parse_result.destructor
    if parse_result.virtual:
        args += parse_result.virtual
    if parse_result.inline:
        args += parse_result.inline
    
    if parse_result.constructor:
        tp = parse_result.constructor[0]
    else:
        tp = parse_result.decl.data_type
    
    if not parse_result.decl:
        parse_result.name = None
    else:
        parse_result.name = parse_result.decl.identifier
    return CppFunctionDeclaration(parse_result.name, tp, parse_result.parameters, args)

def build_function_definition(res):
    return CppFunctionDefinition(res.decl)

def build_complex_type(res):
    return CppComplexTypeDefinition(res.decl.struct_type, res.decl.name)


In [125]:
# C++ Syntax Description

# comments need to be removed
comment = (pp.cStyleComment | pp.cppStyleComment)
preprocessor = pp.lineStart() + pp.Word('#', pp.alphas) + pp.SkipTo( pp.lineEnd() )
preprocessor.setWhitespaceChars(' \r\t')

identifier  = pp.Word( pp.alphas + '_', pp.alphanums + '_' )
persistency = pp.Keyword('static'  ).setParseAction( pp.replaceWith(TypeArgs.STATIC_TYPE)   )
volatility  = pp.Keyword('const'   ).setParseAction( pp.replaceWith(TypeArgs.CONST_TYPE )   ) \
            | pp.Keyword('volatile').setParseAction( pp.replaceWith(TypeArgs.VOLATILE_TYPE) )

reference = pp.Literal('*').setParseAction( pp.replaceWith(CppPointerTypeExpression.POINTER_VAR  ) ) \
          | pp.Literal('&').setParseAction( pp.replaceWith(CppPointerTypeExpression.REFERENCE_VAR) )

# member function on const object
const_function    = pp.Keyword('const'  ).setParseAction( pp.replaceWith(FunctionArgs.CONST_FUNCTION     ) )
virtual_function  = pp.Keyword('virtual').setParseAction( pp.replaceWith(FunctionArgs.VIRTUAL_FUNCTION   ) )
destructor_tag    = pp.Literal('~'      ).setParseAction( pp.replaceWith(FunctionArgs.DESTRUCTOR_FUNCTION) )
abstract_function = (pp.Literal('=') + pp.Literal('0')).setParseAction( pp.replaceWith(FunctionArgs.ABSTRACT_FUNCTION))
inline_function   = pp.Keyword('inline').setParseAction( pp.replaceWith(FunctionArgs.INLINE_FUNCTION) )

complex_type = pp.Keyword('class' ).setParseAction(pp.replaceWith(CppComplexTypeDefinition.CLASS )) \
             | pp.Keyword('struct').setParseAction(pp.replaceWith(CppComplexTypeDefinition.STRUCT)) \
             | pp.Keyword('union' ).setParseAction(pp.replaceWith(CppComplexTypeDefinition.UNION ))

int_value = pp.Optional(pp.Literal('+') | pp.Literal('-'))('sign') + pp.Word(pp.nums)('value')
int_value.setParseAction(lambda res: int(res.value) * ((-1) if res.sign == '-' else 1))
long_value = int_value + pp.Literal('L').suppress()

ref = reference + pp.Optional(volatility)

value_expression = pp.Forward()
#value_expression <<= identifier | 

# all type names
#
# type expressions can be recursive due to templates
# TODO: do not suppress struct, union and class keywords
type_expression = pp.Forward()
type_expression <<= (pp.ZeroOrMore(persistency | volatility).setParseAction(
                    lambda tokens: sum(tokens) # collapse all bitmasks into a single one
                )('args') \
                + pp.Group(
                    (pp.Optional(pp.Keyword('unsigned') | pp.Keyword('signed')) \
                        + (pp.Keyword('double') | pp.Keyword('int') | pp.Keyword('float') \
                        |  pp.Keyword('char') | pp.Keyword('unsigned') | pp.Keyword('signed') | pp.Keyword('void'))).setParseAction( lambda tokens: ' '.join(tokens) ) \
                | (pp.Optional(complex_type).suppress() + identifier + pp.ZeroOrMore(pp.Literal('::') + identifier)).setParseAction( lambda tokens: ''.join(tokens) ))('name') \
                + pp.Optional(
                      pp.Literal('<').suppress() \
                    + csl(type_expression + pp.ZeroOrMore(ref), 1) \
                    + pp.Literal('>').suppress()
                )('template')
).setParseAction( build_type_expression )

# declaration: <type> <address-stars> <var-name> <array-brackets>
var_decl = type_expression('type_id') \
     + pp.ZeroOrMore( pp.Group(ref) )('refs') + identifier('name') \
     + pp.ZeroOrMore( get_scope('[',']') ).suppress() \
     + pp.Optional(pp.Literal('=').suppress() + pp.SkipTo(pp.Literal(';')))
var_decl.setParseAction(lambda res: CppVarDeclaration(build_pointer_type_expression(res.type_id[0], res.refs), res.name))

var_decl_list = type_expression('type_id') \
          + csl(
                pp.Group(
                    pp.ZeroOrMore( pp.Group(ref) )('refs') + identifier('name') \
                  + pp.ZeroOrMore( get_scope('[',']') ).suppress() \
                  + pp.Optional(pp.Literal('=') + pp.SkipTo(pp.Literal(',') | pp.Literal(';')))
                )
          )('ids')
var_decl_list.setParseAction( build_declaration_list )

# parameter list: '(' <list of parameter declarations> or 'void' ')'
# important: '+' has precedence over '|' -> brackets after the '(' literals are semantically needed!
parameter_list = pp.Literal('(').suppress() \
               + pp.Optional( csl(var_decl) | pp.Keyword('void').suppress()) \
               + pp.Literal(')').suppress()

#+ type_expression('ret_type') \
#+ pp.Optional(
#    pp.Group(
#        pp.ZeroOrMore( ref )
#    )('refs') + identifier('name')
#) \

fun_decl = pp.ZeroOrMore(virtual_function('virtual') | inline_function('inline')) \
         + (var_decl('decl') | (pp.Optional(destructor_tag('destructor')) + type_expression('constructor'))) \
         + pp.Group(parameter_list)('parameters') \
         + pp.Optional(const_function('const')) \
         + pp.Optional(abstract_function('abstract'))

fun_decl.setParseAction( build_function )

# TODO: quick and dirty hack to catch function calls
fun_call = identifier + get_scope('(', ')')

fun_def = fun_decl('decl') \
        + pp.Optional(pp.Literal(':') + fun_call + pp.ZeroOrMore(pp.Literal(',').suppress() + fun_call)) \
        + get_scope('{', '}')
fun_def.setParseAction( build_function_definition )

decl = fun_decl | var_decl_list

visibility = pp.Keyword('private'  ).setParseAction( pp.replaceWith( CppComplexTypeDefinition.VISIBILITY_PRIVATE   ) ) \
           | pp.Keyword('public'   ).setParseAction( pp.replaceWith( CppComplexTypeDefinition.VISIBILITY_PUBLIC    ) ) \
           | pp.Keyword('protected').setParseAction( pp.replaceWith( CppComplexTypeDefinition.VISIBILITY_PROTECTED ) )

visibility_space = pp.Optional(visibility + pp.Literal(':').suppress()) + pp.ZeroOrMore(fun_def | (decl + pp.Literal(';').suppress()))

inheritance = pp.Optional(visibility)('visibility') + identifier('base_class_name')
inheritance.setParseAction(lambda res: CppInheritance(res.base_class_name, res.visibility if len(res) != 0 else CppComplexTypeDefinition.VISIBILITY_DEFAULT))

complex_type_decl = complex_type('struct_type') + identifier('name')

type_def = pp.Keyword('typedef') + type_expression('expr') + identifier('name')
type_def.setParseAction(lambda res: CppTypeDefinition(res.expr[0], res.name))

complex_type_def   = pp.Forward()
complex_type_def <<= pp.Optional(visibility) + complex_type_decl('decl') + \
              pp.Optional(pp.Group(pp.Literal(':').suppress() + csl(inheritance)))('base_classes') \
            + pp.Literal('{') \
                + pp.Group(pp.ZeroOrMore(fun_def | (decl + pp.Literal(';').suppress()))) \
                + pp.ZeroOrMore(visibility_space) \
            + pp.Literal('}')
#complex_type_def.setParseAction(build_complex_type)


In [103]:
assert int_value.parseString('-1234')[0] == -1234, 'int-value does not match int'
assert long_value.parseString('-1234L')[0] == -1234, 'long-value does not match long'

try:
    int_value.parseString('1234L', parseAll=True)
except pp.ParseException:
    # all fine!
    pass
else:
    # must not match
    assert False, 'int-value matches long'

try:
    long_value.parseString('1234')
except pp.ParseException:
    # all fine!
    pass
else:
    assert False, 'long-value matches int'


In [104]:
import os
from os.path import join, getsize


In [105]:
# load source file

source_code = '\n'.join(
    open('/home/farad/Programmieren/fg/flightgear/src/Input/FGEventInput.hxx')
)


In [106]:
# remove comments and preprocessor directives from the code
stripped_source = (comment | preprocessor).suppress().transformString(source_code)


In [107]:
printer = CppPrinter()
typeds = type_def.searchString(stripped_source)

for typed in typeds:
    print(printer.typedef_str(typed[0]))


typedef SGSharedPtr<FGEventSetting> FGEventSetting_ptr
typedef std::vector<FGEventSetting_ptr> setting_list_t
typedef SGSharedPtr<FGInputEvent> FGInputEvent_ptr
typedef SGSharedPtr<FGInputDevice> FGInputDevice_ptr


In [108]:
printer = CppPrinter()
#printer.declaration_str(var_decl_list.parseString('double **a')[0])
vd = var_decl_list.parseString('double **a')[0]
printer.declaration_str(vd)

'double ** a'

In [109]:
printer = CppPrinter()
functions = fun_decl.searchString(stripped_source)

print('found %d functions:' % len(functions))

for fun_dcl in functions:
    print(printer.function_decl_str(fun_dcl[0]))
    

found 50 functions:
FGEventData (double aValue, double aDt, int aModifiers)
FGEventSetting (SGPropertyNode_ptr base)
bool Test ()
double GetValue ()
FGInputEvent (FGInputDevice * device, SGPropertyNode_ptr node)
virtual ~FGInputEvent ()
virtual void fire (FGEventData & eventData)
std::string GetName () const
std::string GetDescription () const
virtual void update (double dt)
static FGInputEvent * NewObject (FGInputDevice * device, SGPropertyNode_ptr node)
virtual void fire (SGBinding * binding, FGEventData & eventData)
FGButtonEvent (FGInputDevice * device, SGPropertyNode_ptr node)
virtual void fire (FGEventData & eventData)
FGAxisEvent (FGInputDevice * device, SGPropertyNode_ptr node)
void SetMaxRange (double value)
void SetMinRange (double value)
void SetRange (double min, double max)
virtual void fire (FGEventData & eventData)
FGRelAxisEvent (FGInputDevice * device, SGPropertyNode_ptr node)
virtual void fire (SGBinding * binding, FGEventData & eventData)
FGAbsAxisEvent (FGInputDevic

In [110]:
vdcl = var_decl.searchString('double **a, int x[]')
printer.declaration_str(vdcl[0][0])

'double ** a'

In [111]:
(type_expression + identifier + pp.Optional( get_scope('[',']') )).parseString('binding_list_t bindings[KEYMOD_MAX]', parseAll=True)

([<__main__.CppTypeExpression object at 0x7f0948587c88>, 'bindings', '[', 'KEYMOD_MAX', ']'], {})

In [126]:
printer = CppPrinter()
classes = (complex_type_def | complex_type_decl).searchString(stripped_source)
print('found %d classes:' % len(classes))

for cl in classes:
    print(repr(cl))
    if type(cl[0]) == CppComplexTypeDefinition:
        print(printer.complex_type_str(cl[0]))


found 11 classes:
([2, 'FGEventData', '{', ([<__main__.CppFunctionDefinition object at 0x7f09483efb70>, <__main__.CppVarDeclaration object at 0x7f09485b9a58>, <__main__.CppVarDeclaration object at 0x7f09485a70b8>, <__main__.CppVarDeclaration object at 0x7f09485420b8>], {}), '}'], {'struct_type': [(2, 0)], 'name': [('FGEventData', 1)], 'decl': [(([2, 'FGEventData'], {'struct_type': [(2, 0)], 'name': [('FGEventData', 1)]}), 0)]})
([1, 'FGEventSetting', ([<__main__.CppInheritance object at 0x7f094844e0b8>], {}), '{', ([], {}), 3, ':', <__main__.CppFunctionDeclaration object at 0x7f09484a4fd0>, <__main__.CppFunctionDeclaration object at 0x7f0948556f28>, <__main__.CppFunctionDeclaration object at 0x7f094845a5c0>, 2, ':', <__main__.CppVarDeclaration object at 0x7f0948489b00>, <__main__.CppVarDeclaration object at 0x7f0948556470>, <__main__.CppVarDeclaration object at 0x7f0948489278>, '}'], {'struct_type': [(1, 0)], 'base_classes': [(([([<__main__.CppInheritance object at 0x7f094844e0b8>], {}

In [263]:
dcl = var_decl.searchString('const static unsigned MAX_DEVICES = 1000;')

printer = CppPrinter()
printer.declaration_str(dcl[0][0])

'static const unsigned MAX_DEVICES'

In [264]:
printer = CppPrinter()
test_decl = var_decl_list.parseString('double a', parseAll=True)

for decla in test_decl:
    print(printer.declaration_str(decla))

double a


In [276]:
printer = CppPrinter()
test_fundef = fun_def.parseString('std::string GetName() const { return name; }', parseAll=True)[0]
#print(printer.function_decl_str(test_fundef))
print(repr(test_fundef.name))


''


In [33]:
list(functions[i][0].isConstant() for i in range(len(functions)))

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [34]:
classes = set()
derivations = []

for root, dirs, files in os.walk('/home/farad/Programmieren/flightgear-flightgear/src/'):
    for f in filter((lambda f: f.endswith('.hxx')), files):
        p = join(root, f)
        #print('%s:' % p)
        # file content with comments removed
        file_content = comment.transformString('\n'.join(open(p)))
        for cd in complex_type_def.searchString(file_content):
            classes.add(cd[0])
            if len(cd) > 1:
                base_classes = { e[-1] for e in cd[1:] }
                classes.update(base_classes)
                derivations.extend([ (cd[0], bc) for bc in base_classes ])


IndexError: list index out of range

In [ ]:
derivations

In [70]:
len(classes)

707

In [55]:
import networkx as nx

In [71]:
gr = nx.DiGraph()
gr.add_nodes_from(classes)
gr.add_edges_from(derivations)

In [95]:
nx.write_gexf(gr, '/home/farad/test.gexf')

In [73]:
def write_graph_csv(output_file, graph):
    with open(output_file, 'w') as ostream:
        ostream.write('Source,Target\n')
        for edge in graph.edges_iter():
            ostream.write('%s,%s\n' % edge)


In [74]:
write_graph_csv('/home/farad/test.csv', gr)